In [61]:
from felt import Felt

class FFTFelt(Felt):
    twiddles = [ Felt(x, 337) for x in [1, 85, 148, 111, 336, 252, 189, 226]]

    def __init__(self, val, _prime=None):
        super().__init__(val, 337)

    

In [14]:

FFTFelt(5).twiddles

[Felt(1,337),
 Felt(85,337),
 Felt(148,337),
 Felt(111,337),
 Felt(336,337),
 Felt(252,337),
 Felt(189,337),
 Felt(226,337)]

In [62]:
def fft(vals: list[FFTFelt], domain=FFTFelt.twiddles):
    if len(vals) == 1:
        return vals

    odds = fft(vals[1::2],domain[::2])
    evens = fft(vals[::2],domain[::2])
    ans = [FFTFelt(0)]*len(vals)
    length = len(odds)
    for i, (o,e) in enumerate(zip(odds,evens)):
        ans[i] = e + domain[i] * o
        ans[i+length] = e - domain[i] * o
    return ans


def ifft(vals: list[FFTFelt], domain=FFTFelt.twiddles):
    vals = fft(vals, domain)
    return [x * FFTFelt(len(vals)).inv() for x in [vals[0]] + vals[1:][::-1]]

f = fft([FFTFelt(x) for x in [3,1,4,1,5,9,2,6]])
print(f)
print(ifft(f))


[Felt(31,337), Felt(70,337), Felt(109,337), Felt(74,337), Felt(334,337), Felt(181,337), Felt(232,337), Felt(4,337)]
[Felt(3,337), Felt(1,337), Felt(4,337), Felt(1,337), Felt(5,337), Felt(9,337), Felt(2,337), Felt(6,337)]


In [262]:
from math import log2, sqrt
from random import choice


def fill(poly, domain):
    return poly + [FFTFelt(0) for _ in range(len(domain)-len(poly))]

# FRI Commitment Phase
poly = fill([ FFTFelt(x) for x in [1,2,3,1]], FFTFelt.twiddles)
rounds = int(log2(len(poly)))
evals = fft(poly)

commitment = dict(zip(FFTFelt.twiddles, evals))


test = FFTFelt(189)

def commit(poly, r, domain):
    odds = [ o * r for o in poly[1::2] ]
    evens = poly[::2]
    p = fill([ o + e for (o,e) in zip(odds,evens)], domain)
    return (p, fft(p, domain))

domain = FFTFelt.twiddles
#3z+1 and Pi,1(z) = z+2


r1 = FFTFelt.random()
domain = domain[::2]
(poly1,evals) = commit(poly, r1, domain)
commitment2 = dict(zip(domain, evals))

r2 = FFTFelt.random()
domain = domain[::2]
(poly2, evals) = commit(poly1, r2, domain)
commitment3 = dict(zip(domain, evals))

s0 = choice(FFTFelt.twiddles)
s1 = s0**2
s2 = s1**2

#G1(s1) = (x0 − s0) · (s′0 − s0)−1 · G0(s′0) + (x0 − s′0) · (s0 − s′0)−1 · G0(s0).
half = len(FFTFelt.twiddles)//2
sprime = { x: y for (x,y) in zip(FFTFelt.twiddles, FFTFelt.twiddles[half:]+FFTFelt.twiddles[:half])}

#g1 = ()
# Gi+1(si+1) = (xi − si) · (s′i − si)−1 · Gi(s′i) + (xi − s′i) · (si − s′i)−1 · Gi(si).
# p0(v) = g0(v2) + v · h0(v2) 
# p0(−v) = g0(v2) − v · h0(v2)
p0 = commitment[s0]
p0_ = commitment[sprime[s0]]
s0_ = sprime[s0]
x = s0

def next(si, si_, pi, pi_, xi):
    return (xi-si)*(si_-si).inv() * pi_+(xi - si_)*(si -si_).inv() * pi
s1_ = sprime[s1]
s2_ = sprime[s2]

p1 = commitment2[s1]
p1_ = commitment2[sprime[s1]]
#print(next(s0,s0_,p0,p0_,r1),p1)
assert(next(s0,s0_,p0,p0_,r1).val == p1.val)
p2 = commitment3[s2]
p2_ = commitment3[sprime[s2]]
#print(p2, commitment3, r2)
#print(next(s1,s1_,p1,p1_,r2))
assert(next(s1,s1_,p1,p1_,r2) == p2)



In [261]:
1+407*2**119

270497897142230380135924736767050121217

In [276]:
modulus = 0xFFFFFFFF00000001
g = 1753635133440165772

In [289]:
modulus
gen = Felt(g, modulus)
root = gen**(2**23)

In [290]:
for i in range(8):
    root = root**2
    print(root)

print(Felt(17870292113338400769,18446744069414584321) * Felt(18446744069397807105,18446744069414584321))

Felt(13797081185216407910,18446744069414584321)
Felt(17870292113338400769,18446744069414584321)
Felt(549755813888,18446744069414584321)
Felt(70368744161280,18446744069414584321)
Felt(17293822564807737345,18446744069414584321)
Felt(18446744069397807105,18446744069414584321)
Felt(281474976710656,18446744069414584321)
Felt(18446744069414584320,18446744069414584321)
Felt(18446744035054321673,18446744069414584321)


In [149]:
from math import ceil, log, log2
from random import choice
from felt import Felt

def zero_pad(poly, domain):
    return poly + [GLFelt(0) for _ in range(len(domain)-len(poly))]
    
def next(si, si_, pi, pi_, xi):
    return (xi-si)*(si_-si).inv() * pi_+(xi - si_)*(si -si_).inv() * pi 

class GLFelt(Felt):
    """ Goldilocks Feild 2^64-2-32+1 with 2^32 roots """
    def __init__(self, val, _=None):
        super().__init__(val,0xFFFFFFFF00000001)
    
    def roots_of_unity(n):
        root = GLFelt(1753635133440165772)
        order = 2**32
        while order != n:
            root = root**2
            order = order/2
        return [ root ** x for x in range(n) ] 
    
    def fft(vals, domain):
        if len(vals) == 1:
            return vals

        odds = GLFelt.fft(vals[1::2],domain[::2])
        evens = GLFelt.fft(vals[::2],domain[::2])
        ans = [GLFelt(0)]*len(vals)
        length = len(odds)
        for i, (o,e) in enumerate(zip(odds,evens)):
            ans[i] = e + domain[i] * o
            ans[i+length] = e - domain[i] * o
        return ans

class Fri:
    def __init__(self, rate, queries, poly):
        self.poly = poly
        self.degree = len(poly)
        group_size = pow(2, ceil(log(int(1/rate * self.degree))/log(2)))
        self.l = GLFelt.roots_of_unity(group_size)
        half = len(self.l) // 2
        self.s_prime = { x: y for (x,y) in zip(self.l, self.l[half:]+self.l[:half]) }
        self.commits = []
        self.rs = []
        self.queries = queries

    def execute(self):
        rounds = int(log2(len(self.poly)))
        poly = self.poly
        domain = self.l
        self.commits.append(dict(zip(domain, GLFelt.fft(zero_pad(poly,domain), domain))))
        for _ in range(rounds):
            r = GLFelt.random()
            self.rs.append(r)
            domain = domain[::2]
            poly = self.commit(poly, r, domain)

        for _ in range(self.queries):
            s = choice(self.l)
            s_ = self.s_prime[s]
            for i in range(rounds):
                p = self.commits[i][s]
                p_ = self.commits[i][s_]
                s_2 = s**2
                assert(next(s,s_,p,p_,self.rs[i]) == self.commits[i+1][s_2])
                s = s_2
                s_ = self.s_prime[s]
    
    def commit(self, poly, r, domain):
        odds = [ o * r for o in poly[1::2] ]
        evens = poly[::2]
        p = zero_pad([ o + e for (o,e) in zip(odds,evens)], domain)
        self.commits.append(dict(zip(domain, GLFelt.fft(p, domain))))
        return p
        

Fri(.4, 50, [GLFelt(17),GLFelt(2),GLFelt(3),GLFelt(4), GLFelt(14), GLFelt(27)]).execute()

In [12]:

GLFelt(0).prime

18446744069414584321